In [53]:
# Question 1

from nltk.corpus.reader import BracketParseCorpusReader
from nltk.lm import Vocabulary
corpus = BracketParseCorpusReader(root="ressources", fileids=["p1_train_Thomas.txt"])
text = corpus.words()

vocab = Vocabulary(text, unk_cutoff=3)
vocab_ten_last = sorted(vocab)[-10:]
ret_array = [(0, 0)]*10
for i in range(10):
    word = vocab_ten_last[i]
    ret_array[i] = (word, vocab[word])
print(ret_array)

print("\n====================================================================================================\n")

count_unk = 0
for word in text:
    if word not in vocab:
        count_unk += 1
oov = (count_unk/len(text))*100
    
print("{:.3f}".format(oov))

[('wry', 6), ('year', 32), ('years', 8), ('yes', 3), ('yet', 11), ('you', 166), ('young', 12), ('your', 31), ('yourself', 5), ('zippy', 3)]


19.295


In [52]:
# Question 2

from nltk.corpus.reader import BracketParseCorpusReader
from nltk.lm.preprocessing import pad_both_ends
from nltk.util import bigrams
from nltk.lm import Vocabulary
import numpy as np

# import time
# t1 = time.time()

corpus = BracketParseCorpusReader(root="ressources", fileids=["p1_train_Thomas.txt"])
texts = corpus.sents()

sentence_10_bi = list(bigrams(list(pad_both_ends(texts[9], n=2))))
print("Question 1:")
print(sentence_10_bi)

print("\n====================================================================================================\n")
print("Question 2:")

text = corpus.words()
vocab = Vocabulary(text, unk_cutoff=3)

mle_pers = {}
for sentence in range(len(texts)):
    for bi in list(bigrams(list(pad_both_ends(texts[sentence], n=2)))):
        if bi[0] == '<s>':
            bi = (bi[0], vocab.lookup(bi[1]))
        elif bi[1] == '</s>':
            bi = (vocab.lookup(bi[0]), bi[1])
        else:
            bi = (vocab.lookup(bi[0]), vocab.lookup(bi[1]))
        if mle_pers.get(bi[0], -1) == -1:
            mle_pers[bi[0]] = {}
        count_bi = mle_pers[bi[0]].get(bi[1], 0)
        mle_pers[bi[0]][bi[1]] = count_bi+1

count_nexts = {}
key_nexts = {}
for first_word in mle_pers.keys():    
    count_next = 0
    for i in mle_pers[first_word]:
        count_next += mle_pers[first_word][i]
    count_nexts[first_word] = count_next
    for second_word in mle_pers[first_word].keys():
        previous = mle_pers[first_word][second_word]
        mle_pers[first_word][second_word] = previous/count_nexts[first_word]
        
    key_next = list(mle_pers[first_word].keys())
    key_next = sorted(key_next, key= mle_pers[first_word].__getitem__)
    key_nexts[first_word] = key_next   

ret_dict_mle = {}
ret_dict_mle[key_nexts["<s>"][-1]] = mle_pers["<s>"][key_nexts["<s>"][-1]]  #"{:.4f}".format()
ret_dict_mle[key_nexts["<s>"][-2]] = mle_pers["<s>"][key_nexts["<s>"][-2]]
ret_dict_mle[key_nexts["<s>"][-3]] = mle_pers["<s>"][key_nexts["<s>"][-3]]
print(ret_dict_mle)

print("\n====================================================================================================\n")
print("Question 3:")

mle_pers = {}
        
for sentence in range(len(texts)):
    for bi in list(bigrams(list(pad_both_ends(texts[sentence], n=2)))):
        if bi[0] == '<s>':
            bi = (bi[0], vocab.lookup(bi[1]))
        elif bi[1] == '</s>':
            bi = (vocab.lookup(bi[0]), bi[1])
        else:
            bi = (vocab.lookup(bi[0]), vocab.lookup(bi[1]))
        if mle_pers.get(bi[0], -1) == -1:
            mle_pers[bi[0]] = {}
        count_bi = mle_pers[bi[0]].get(bi[1], 1)
        mle_pers[bi[0]][bi[1]] = count_bi+1

key_nexts = {}
len_vocab = len(vocab)
for first_word in mle_pers.keys():
    for second_word in mle_pers[first_word].keys():
        previous = mle_pers[first_word][second_word]
        mle_pers[first_word][second_word] = previous/(count_nexts[first_word] + len_vocab + 3)
    key_next = list(mle_pers[first_word].keys())
    key_next = sorted(key_next, key= mle_pers[first_word].__getitem__)
    key_nexts[first_word] = key_next

ret_dict_mle = {}
ret_dict_mle[key_nexts["<s>"][-1]] = mle_pers["<s>"][key_nexts["<s>"][-1]]  #"{:.4f}".format()
ret_dict_mle[key_nexts["<s>"][-2]] = mle_pers["<s>"][key_nexts["<s>"][-2]]
ret_dict_mle[key_nexts["<s>"][-3]] = mle_pers["<s>"][key_nexts["<s>"][-3]]
print(ret_dict_mle)

print("\n====================================================================================================\n")
print("Question 4:")

test_set = BracketParseCorpusReader(root="ressources", fileids=["p1_test_Thomas.txt"]).sents()

mult_score = 0
M = 0
for sentence in range(len(test_set)):
    for bi in list(bigrams(list(pad_both_ends(test_set[sentence], n=2)))):
        if bi[0] == '<s>':
            bi = (bi[0], vocab.lookup(bi[1]))
        elif bi[1] == '</s>':
            bi = (vocab.lookup(bi[0]), bi[1])
        else:
            bi = (vocab.lookup(bi[0]), vocab.lookup(bi[1]))
        mult_score += np.log2(mle_pers[bi[0]].get(bi[1], 1/(count_nexts[bi[0]] + len_vocab + 3)))
        M += 1
        
LL = mult_score/M
PP = 2**(-LL)
print("{:.3f}".format(PP))

# t2 = time.time()
# print("temps pris : "+ str((t2-t1)))

Question 1:
[('<s>', 'It'), ('It', "'s"), ("'s", 'a'), ('a', 'coming-of-age'), ('coming-of-age', 'story'), ('story', 'we'), ('we', "'ve"), ("'ve", 'all'), ('all', 'seen'), ('seen', 'bits'), ('bits', 'of'), ('of', 'in'), ('in', 'other'), ('other', 'films'), ('films', '--'), ('--', 'but'), ('but', 'it'), ('it', "'s"), ("'s", 'rarely'), ('rarely', 'been'), ('been', 'told'), ('told', 'with'), ('with', 'such'), ('such', 'affecting'), ('affecting', 'grace'), ('grace', 'and'), ('and', 'cultural'), ('cultural', 'specificity'), ('specificity', '.'), ('.', '</s>')]


Question 2:
{'<UNK>': 0.268, 'The': 0.1105, 'A': 0.0915}


Question 3:
{'<UNK>': 0.14486107364445644, 'The': 0.05988670083625573, 'A': 0.04963582411653628}


Question 4:
167.333


In [59]:
# Question 3

from nltk.corpus.reader import BracketParseCorpusReader
from nltk.lm import Vocabulary
import numpy as np

corpus = BracketParseCorpusReader(root="ressources", fileids=["p1_train_Thomas.txt"])
parsed_texts = corpus.parsed_sents()
vocab = Vocabulary(corpus.words(), unk_cutoff=3)

N_neg = 0
N_pos = 0
N_doc = len(parsed_texts)
bag_neg = {}
bag_pos = {}
for sentence in range(N_doc):
    sequence = parsed_texts[sentence]
    if int(sequence.label()) < 2:
        N_neg += 1
        for word in sequence.leaves():
            bag_neg[word] = bag_neg.get(word, 0) + 1
    else:
        N_pos += 1
        for word in sequence.leaves():
            bag_pos[word] = bag_pos.get(word, 0) + 1

count_neg = 0
for word in bag_neg.keys():
    count_neg += bag_neg[word]
count_pos = 0
for word in bag_pos.keys():
    count_pos += bag_pos[word]
    
test = BracketParseCorpusReader(root="ressources", fileids=["p1_test_Thomas.txt"])
test_set = test.sents()

test_pred = [0]*len(test_set)
vocab_len = len(vocab) + 2
for sentence in range(len(test_set)):
    neg_score = 0
    pos_score = 0
    for word in test_set[sentence]:
        if word in vocab:
            neg_score += np.log( (bag_neg.get(word, 0) + 1) / (count_neg + vocab_len) )
            pos_score += np.log( (bag_pos.get(word, 0) + 1) / (count_pos + vocab_len) )
    
    neg_score += np.log((N_neg/N_doc))    # Ajout du prior avec les logs
    pos_score += np.log((N_pos/N_doc))
    if neg_score > pos_score:
        test_pred[sentence] = 0
    else:
        test_pred[sentence] = 1

test_verif = test.parsed_sents()
ratio = 0
for sentence in range(len(test_set)):
    y = int(test_verif[sentence].label())
    if (test_pred[sentence] == (y>=2)):
        ratio += 1
print((ratio/len(test_set))*100)

64.0


In [60]:
# Question 4
# 1) Binary Naive Bias

from nltk.corpus.reader import BracketParseCorpusReader
from nltk.lm import Vocabulary
import numpy as np

corpus = BracketParseCorpusReader(root="ressources", fileids=["p1_train_Thomas.txt"])
parsed_texts = corpus.parsed_sents()
vocab = Vocabulary(corpus.words(), unk_cutoff=3)

N_neg = 0
N_pos = 0
N_doc = len(parsed_texts)
bag_neg = {}
bag_pos = {}
for sentence in range(N_doc):
    sequence = parsed_texts[sentence]
    words_seen = []
    if int(sequence.label()) < 2:
        N_neg += 1
        for word in sequence.leaves():
            if word not in words_seen:
                bag_neg[word] = bag_neg.get(word, 0) + 1
                words_seen.append(word)
    else:
        N_pos += 1
        for word in sequence.leaves():
            if word not in words_seen:
                bag_pos[word] = bag_pos.get(word, 0) + 1
                words_seen.append(word)

count_neg = 0
for word in bag_neg.keys():
    count_neg += bag_neg[word]
count_pos = 0
for word in bag_pos.keys():
    count_pos += bag_pos[word]
    
test = BracketParseCorpusReader(root="ressources", fileids=["p1_test_Thomas.txt"])
test_set = test.sents()

test_pred = [0]*len(test_set)
vocab_len = len(vocab) + 2
for sentence in range(len(test_set)):
    neg_score = 0
    pos_score = 0
    words_seen = []
    for word in test_set[sentence]:
        if word in vocab and word not in words_seen:
            neg_score += np.log( (bag_neg.get(word, 0) + 1) / (count_neg + vocab_len) )
            pos_score += np.log( (bag_pos.get(word, 0) + 1) / (count_pos + vocab_len) )
            words_seen.append(word)
    
    neg_score += np.log((N_neg/N_doc))
    pos_score += np.log((N_pos/N_doc))
    if neg_score > pos_score:
        test_pred[sentence] = 0
    else:
        test_pred[sentence] = 1

test_verif = test.parsed_sents()
ratio = 0
for sentence in range(len(test_set)):
    y = int(test_verif[sentence].label())
    if (test_pred[sentence] == (y>=2)):
        ratio += 1
print((ratio/len(test_set))*100)

66.5


In [2]:
# Question 4
# 2) negative

from nltk.corpus.reader import BracketParseCorpusReader
from nltk.lm import Vocabulary
import numpy as np

corpus = BracketParseCorpusReader(root="ressources", fileids=["p1_train_Thomas.txt"])
parsed_texts = corpus.parsed_sents()
vocab = Vocabulary(corpus.words(), unk_cutoff=3)
negate = ["n't", "not", "no", "never"]
punct = ['.', ',', ':', '?', '!']

N_neg = 0
N_pos = 0
N_doc = len(parsed_texts)
bag_neg = {}
bag_pos = {}
neg_words_seen = set()
for sentence in range(N_doc):
    sequence = parsed_texts[sentence]
    in_neg = False
    if int(sequence.label()) < 2:
        N_neg += 1
        for word in sequence.leaves():
            if word in punct: in_neg = False
            if in_neg: 
                if word in vocab:
                    neg_words_seen.add(word+"_NOT")
                word += "_NOT"
            bag_neg[word] = bag_neg.get(word, 0) + 1
            if word in negate: in_neg = True
    else:
        N_pos += 1
        for word in sequence.leaves():
            if word in punct: in_neg = False
            if in_neg: 
                if word in vocab:
                    neg_words_seen.add(word+"_NOT")
                word += "_NOT"
            bag_pos[word] = bag_pos.get(word, 0) + 1
            if word in negate: in_neg = True


count_neg = 0
for word in bag_neg.keys():
    count_neg += bag_neg[word]
count_pos = 0
for word in bag_pos.keys():
    count_pos += bag_pos[word]
    
test = BracketParseCorpusReader(root="ressources", fileids=["p1_test_Thomas.txt"])
test_set = test.sents()


test_pred = [0]*len(test_set)
vocab_len = len(vocab) + 2
for sentence in range(len(test_set)):
    neg_score = 0
    pos_score = 0
    in_neg = False
    for word in test_set[sentence]:
        if word in punct: in_neg = False
        if in_neg:
            word += "_NOT"
        if word in vocab or word in neg_words_seen:
            neg_score += np.log( (bag_neg.get(word, 0) + 1) / (count_neg + vocab_len) )
            pos_score += np.log( (bag_pos.get(word, 0) + 1) / (count_pos + vocab_len) )
        if word in negate: in_neg = True
    
    neg_score += np.log((N_neg/N_doc))
    pos_score += np.log((N_pos/N_doc))
    if neg_score > pos_score:
        test_pred[sentence] = 0
    else:
        test_pred[sentence] = 1

test_verif = test.parsed_sents()
ratio = 0
for sentence in range(len(test_set)):
    y = int(test_verif[sentence].label())
    if (test_pred[sentence] == (y>=2)):
        ratio += 1
print((ratio/len(test_set))*100)

65.5
